# Working with Data - Computer Lab for Guest Lecture Julia Lane

In this computer lab we will learn more more details and practice data work to enhance the content of the lecture presented by Julia Lane on responsible data use. We will address a research question, think about data and measurement errors, and manipulate data. 

OUTLINE: 
1. Define a research question 
2. Think about what data are available 
3. Think about possible measurement errors 
4. Think about the interpretation of your results 
5. Inform your results by linking datasets 

# 1. Define a reserach question
Which Community Districts in NYC show the highest number of complaints?

# 2. Think about what data are available
Find suitable data by searching the CUSP Data Catalog https://datahub.cusp.nyu.edu/catalog. You can use Urban Profiler to investigate the Metadata associated with each dataset. Using this tool will help you to decide which attributes of the data you need to answer your question so you don't have to load the entire dataset. 

In [1]:
import os
import pandas as pd
import numpy as np
import re
PUIdata = os.getenv('PUIDATA')

In [ ]:
# Load dataset
data = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9')

In [ ]:
data.columns

# 3. Think about possible measurement errors
Do you see any problems regarding possible measurement error? Think about who is represented in the data, ommissions, duplications, content error, missing data, etc. 

In [ ]:
# Check if all Boroughs and Community Districts are represented in the Data 
data['Community Board'].count()

In [ ]:
# How many unique values do we have? 
# we have 6 unique Boroughs and 77 Community Dsitricts in the data
data = data.sort_values(by = 'Community Board')


In [ ]:
len(data['Borough'].unique())

In [ ]:
len(data['Community Board'].unique())

In [ ]:
# Why do we have so many? Some of them are unspecified, missing. Some might be invalid entries. 
# We should have 59 Community Districts.
#http://www1.nyc.gov/site/planning/community/community-portal.page

In [ ]:
community = np.unique(data['Community Board'].values)[1:60]

In [ ]:
# Check for duplicates? Are these plausible?
data2 = pd.merge(data, pd.DataFrame({'community':community}), 
                 left_on='Community Board', right_on='community', how='inner')

In [ ]:
data2.head()

In [ ]:
data2.duplicated().shape

In [ ]:
# What about missing values? Can you detect any patterns? 
data3 = data2[data2.Location.isin([np.nan])==True]
# There are 85468 rows with missing 'Location' Value.
data3.shape

In [ ]:
# Data Editing: Is it possible to replace missing values? Is it possible to use Complaint Type 
# to fill missings in Resolution Descriptor?
data4 = data2[data2['Resolution Description'].isin([np.nan])==True]

In [ ]:
data4.shape

In [ ]:
# Generate marker for unplausible Community Districts
# How do these districts look like? 
del data2['Resolution Description']
# I drop the rows with NaN value 
data2 = data2.dropna()


In [ ]:
# Here's the counts of the number of complaints in each community board.
data2_CB = data2.groupby(data2['Community Board']).agg(count_nonzero)

In [ ]:
# Drop the marked districts

In [ ]:
# Produce your result: Generate an indicator which ranks the Community District by complaint numbers 
# on the Community district level
CB_Rank = pd.DataFrame(data2_CB.sort(columns=['Unique Key'], ascending= False).index)
CB_Rank['Rank'] = range(1,60,1)
# Table below shows 59 community boards and their corresponding rank of the number of complaints.
CB_Rank

In [ ]:
# Safe reduced data frame (Community District level)

# 4. Think about the interpretation of your results?
What do you have to keep in mind when interpreting your results? Are they generable? Does the way the data is collected influence your results? To better inform city agancies it might be good to explore in more detail the underlying dempgraphics/infrastructure of a Community District becasue this might influence 311 calls. You can do this by merging external data on the Community District level to your analysis data. 

In [ ]:
# Population by Community District
#df_pop = pd.read_csv(PUIdata + "/Final_Demographics.csv")
df_pop = pd.read_csv('http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv')

In [ ]:
# Check variables in file
df_pop.columns

In [ ]:
df_pop.head()

In [ ]:
# How many community districts are in file? 
print len(df_pop['cd_id'].unique())
df_pop['cd_id'].unique()

In [ ]:
# Manipulate data to get some information on demographics by Community District. 
# Think about who might be more likely to call 311
df_pop_density = df_pop.loc[:,['cd_id','Population Density (per sq. mile)']]
df_pop_density = df_pop_density.sort(columns=['Population Density (per sq. mile)'], ascending=False)
df_pop_density.columns.values[0] = "CD_ID"
df_pop_density.set_value(15, 'CD_ID', 'MN11')
df_pop_density 

In [ ]:
# Save data frame

In [ ]:
# Infrastructure by Community District
df_infr = pd.read_csv(PUIdata + "/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [ ]:
# Check variables in file
df_infr.shape

In [ ]:
# How many community districts are in file? 
(df_infr['Qualifying Name']).unique()

In [ ]:
# Manipulate data to get some information on internet/broadband useage by Community District
# Aggregate the mobile subscription data
df_infr.ix[:,['Qualifying Name','Households: With Mobile Broadband','Households: With Mobile Broadband.1',
              'Households: With Mobile Broadband.2', 'Households: With Mobile Broadband.3',
             'Households: With Mobile Broadband.4', 'Households: Mobile Broadband Alone or With Dialup']].head()

In [ ]:
df_infr['Mobile Subscription'] = df_infr[['Households: With Mobile Broadband',
                                              'Households: With Mobile Broadband.1',
                                              'Households: With Mobile Broadband.2', 
                                              'Households: With Mobile Broadband.3',
                                              'Households: With Mobile Broadband.4', 
                                              'Households: Mobile Broadband Alone or With Dialup']].sum(axis=1)

In [ ]:
df_infr[['Qualifying Name','Mobile Subscription']]
df_infr_mobile = df_infr[['Qualifying Name','Mobile Subscription']]
df_infr_mobile = df_infr_mobile.drop_duplicates(subset = "Qualifying Name")
df_infr_mobile = df_infr_mobile.sort(columns= ['Mobile Subscription'], ascending=False)

In [ ]:
# Aggregate internet type by high and low connections
df_infr.columns
df_infr_internet = df_infr[['Qualifying Name', 'Households: Dial-Up Alone', 'Households: Dsl', 'Households: Cable Modem',
        'Households: Fiber-Optic', 'Households: Satellite Internet Service', 
         'Households: Two or More Fixed Broadband Types, or Other', 
         'Households: Mobile Broadband Alone or With Dialup']]

In [ ]:
# Save data frame 

# 5. Inform your results by linking datasets
Now you want to link the three data frames to produce summary statistics for Community Districts which show a high number of complaints vs. Community Districts which show a lower number of complaints. Please keep in mind that the identifiers used for the linkage (Community Dostrict IDs) should be recored the same way. Use regular expressions to harmonize the identifiers if possible. The identifiers should look like BK01, BK02, etc.
https://docs.python.org/2/library/re.html

In [ ]:
# Harmonize identifier of dataframe 1
for i in xrange(CB_Rank.shape[0]):
    s = ""
    if CB_Rank['Community Board'].values[i][3:] == 'MANHATTAN':
        s='{}{}'.format('MN', CB_Rank['Community Board'].values[i][:3])
    elif CB_Rank['Community Board'].values[i][3:] == 'BROOKLYN':
        s='{}{}'.format('BK', CB_Rank['Community Board'].values[i][:3])
    elif CB_Rank['Community Board'].values[i][3:] == 'BRONX':
        s='{}{}'.format('BX', CB_Rank['Community Board'].values[i][:3])
    elif CB_Rank['Community Board'].values[i][3:] == 'QUEENS':
        s='{}{}'.format('QN', CB_Rank['Community Board'].values[i][:3])
    elif CB_Rank['Community Board'].values[i][3:] == 'STATEN ISLAND':
        s='{}{}'.format('SI', CB_Rank['Community Board'].values[i][:3])
    CB_Rank.loc[i,'CD_ID'] = s.strip()
    

In [ ]:
CB_Rank = CB_Rank.loc[:,['Rank', 'CD_ID']]

In [ ]:
# Harmonize identifier of dataframe 2

In [ ]:
# Harmonize identifier of dataframe 3

In [ ]:
bx0102 = df_infr_mobile[df_infr_mobile['Qualifying Name'] == \
                    'NYC-Bronx Community District 1 & 2--Hunts Point, New York']['Mobile Subscription'].values[0]/2
BX01 = {"Qualifying Name": "BX01", 'Mobile Subscription': bx0102}
BX02 = {"Qualifying Name": "BX02", 'Mobile Subscription': bx0102}

bx0306 =df_infr_mobile[df_infr_mobile['Qualifying Name'] == \
                    'NYC-Bronx Community District 3 & 6--Belmont, New York']['Mobile Subscription'].values[0]/2
BX03 = {"Qualifying Name": "BX03", 'Mobile Subscription': bx0306}
BX06 = {"Qualifying Name": "BX06", 'Mobile Subscription': bx0306}

mn0102 = df_infr_mobile[df_infr_mobile['Qualifying Name'] == \
                    'NYC-Manhattan Community District 1 & 2--Battery Park City, New York']['Mobile Subscription'].values[0]/2 
MN01 = {"Qualifying Name": "MN01", 'Mobile Subscription': mn0102}
MN02 = {"Qualifying Name": "MN02", 'Mobile Subscription': mn0102}

mn0405 = df_infr_mobile[df_infr_mobile['Qualifying Name'] == \
                    'NYC-Manhattan Community District 4 & 5--Chelsea, New York']['Mobile Subscription'].values[0]/2 
MN04 = {"Qualifying Name": "MN04", 'Mobile Subscription': mn0405}
MN05 = {"Qualifying Name": "MN05", 'Mobile Subscription': mn0405}

In [ ]:
df_sperate = pd.DataFrame([BX01, BX02, BX03, BX06, MN01, MN02, MN04, MN05])

In [ ]:
CD_id = []
for i in xrange(df_infr_mobile.shape[0]):
    s = ""
    num = re.search('....(?<=--)', df_infr_mobile['Qualifying Name'].values[i]).group(0)[:-2]
    if len(num.strip())==1:
        num = '0'+num.strip()
    if re.search('(?<=-)\w+', df_infr_mobile['Qualifying Name'].values[i]).group(0) == 'Manhattan':
        s='{}{}'.format('MN',num )
    elif re.search('(?<=-)\w+', df_infr_mobile['Qualifying Name'].values[i]).group(0) == 'Brooklyn':
        s='{}{}'.format('BK', num)
    elif re.search('(?<=-)\w+', df_infr_mobile['Qualifying Name'].values[i]).group(0) == 'Bronx':
        s='{}{}'.format('BX', num)
    elif re.search('(?<=-)\w+', df_infr_mobile['Qualifying Name'].values[i]).group(0) == 'Queens':
        s='{}{}'.format('QN', num)
    elif re.search('(?<=-)\w+', df_infr_mobile['Qualifying Name'].values[i]).group(0) == 'Staten':
        s='{}{}'.format('SI', num)
    CD_id.append(s)
df_infr_mobile['Qualifying Name'] = CD_id

In [ ]:
df_infr_mobile_merge = pd.merge(left=df_infr_mobile, right=df_sperate, on="Qualifying Name",how='outer')

In [ ]:
for i in xrange(df_infr_mobile_merge.shape[0]):
    if np.isnan(df_infr_mobile_merge.loc[i, 'Mobile Subscription_y']):
        df_infr_mobile_merge.loc[i, 'Mobile Subscription'] = df_infr_mobile_merge.loc[i, 'Mobile Subscription_x']
    else:
        df_infr_mobile_merge.loc[i, 'Mobile Subscription'] = df_infr_mobile_merge.loc[i, 'Mobile Subscription_y']

In [ ]:
df_infr_mobile_final = df_infr_mobile_merge.loc[:, ['Qualifying Name', 'Mobile Subscription']]
df_infr_mobile_final.columns.values[0]='CD_ID'
print df_infr_mobile_final.shape
df_infr_mobile_final.sort_values(by=['Mobile Subscription'], ascending=False)

In [ ]:
df1 = pd.merge(CB_Rank, df_infr_mobile_final, on='CD_ID',how='inner')

In [ ]:
df2 = df_pop_density.reset_index()
df2 = df2.iloc[:,[1,2]]

In [ ]:
df = pd.merge(df1, df2, on='CD_ID', how='inner')

In [ ]:
# Link the 3 dataframes

In [ ]:
# Are the demographics and infrastructure different in Community Districts that show more complaints than others?
pl.plot(df.Rank, df['Population Density (per sq. mile)'],'o-g', label="Population Density")
pl.plot(df.Rank, df['Mobile Subscription'],'o-r',label='Mobile Subscription')
pl.xlabel("Rank")
pl.title( "Complaint Rank\n V.S. \n Mobile Subscription and Population Density")
pl.legend(loc = 'best')